# RANK function

The `RANK()` function is a window function in SQL that assigns a unique rank to each row within a partition of a result set. The rank is determined based on the order specified in the `ORDER BY` clause within the `OVER()` clause. Here's a breakdown of the syntax:

### `RANK() OVER(...)`

- **`RANK()`**: This is the function that assigns a rank to each row. Rows with the same values in the order by clause will receive the same rank.

- **`OVER(...)`**: This clause defines how the ranking is calculated. Inside the `OVER()` clause, you can specify two key things:
  1. **Partitioning (`PARTITION BY`)** (optional): Divides the result set into partitions and the ranking is calculated within each partition. If omitted, the ranking is done across the entire result set.
  2. **Ordering (`ORDER BY`)**: Defines the order in which the rows are ranked. The rows are ordered by the columns specified here, and the rank is assigned according to this order.

### Example:

```sql
RANK() OVER(ORDER BY home_goal + away_goal DESC) AS ranking
```

- **`ORDER BY home_goal + away_goal DESC`**: This orders the rows based on the total of `home_goal` and `away_goal`, from the highest to the lowest. The `DESC` keyword specifies descending order.

- **`AS ranking`**: This assigns the output of the `RANK()` function to a column named `ranking`.

### How `RANK()` Works:
- If two rows have the same value for `home_goal + away_goal`, they will receive the same rank.
- The next row after the tied ranks will have a rank that skips ahead by the number of tied rows. For example, if two rows tie for rank 1, the next row will be ranked 3, not 2.

### Difference Between `RANK()` and Other Functions:
- **`DENSE_RANK()`**: Similar to `RANK()`, but does not skip ranks after ties. If two rows tie for rank 1, the next rank will be 2.
- **`ROW_NUMBER()`**: Assigns a unique, sequential integer to rows without regard to ties, so every row gets a distinct number.

This is particularly useful when you want to rank results where the order of importance is defined by certain criteria (e.g., the total goals in your case), and you need to handle ties appropriately.

### Example: rank soccer matches in the 2011/2012 season by the total of home_goal and away_goal, ordered from highest to lowest.

In [1]:
SELECT date,
       home_goal, 
       away_goal,
       RANK() OVER(ORDER BY home_goal + away_goal DESC) AS ranking
FROM match_data_2011_2012.csv
WHERE season = '2011/2012'
LIMIT 5;

,date,home_goal,away_goal,ranking
0,2012-03-10 00:00:00+00:00,7,1,1
1,2011-12-09 00:00:00+00:00,5,2,2
2,2012-05-12 00:00:00+00:00,2,5,2
3,2011-10-23 00:00:00+00:00,3,4,2
4,2011-12-19 00:00:00+00:00,3,4,2


This query ranks the matches based on the sum of home_goal and away_goal, with the highest total getting the rank of 1.

# Running total

A window function for calculating a running total is typically done using the `SUM()` function along with the `OVER()` clause in SQL. This function calculates a cumulative sum (or running total) over a set of rows that is defined by the window. Here's a breakdown of the syntax:

### General Syntax:

```sql
SUM(column_name) OVER(
    [PARTITION BY partition_column]
    ORDER BY order_column
) AS running_total
```

### Components:

1. **`SUM(column_name)`**: This is the aggregation function that calculates the sum of values in `column_name`.

2. **`OVER(...)`**: This clause defines the window over which the `SUM()` function is applied. The `OVER()` clause can include:
   - **`PARTITION BY partition_column`** (optional): Divides the result set into partitions (or groups) and calculates the running total within each partition. If omitted, the running total is calculated across the entire result set.
   - **`ORDER BY order_column`**: Specifies the order in which the rows are processed for the running total. The running total is calculated in this order.

### Example:

Let's say we have a table `sales` with the following columns: `date`, `sales_amount`. We want to calculate the running total of `sales_amount` ordered by `date`:

```sql
SELECT 
    date,
    sales_amount,
    SUM(sales_amount) OVER(ORDER BY date) AS running_total
FROM sales;
```

### How It Works:

- **`SUM(sales_amount) OVER(ORDER BY date)`**: This calculates a cumulative sum of `sales_amount` for each row, starting from the earliest `date` and adding up as it goes through each subsequent row.

### Example Output:

Given this `sales` table:

| date       | sales_amount |
|------------|--------------|
| 2024-01-01 | 100          |
| 2024-01-02 | 150          |
| 2024-01-03 | 200          |
| 2024-01-04 | 50           |

The result of the query would be:

| date       | sales_amount | running_total |
|------------|--------------|---------------|
| 2024-01-01 | 100          | 100           |
| 2024-01-02 | 150          | 250           |
| 2024-01-03 | 200          | 450           |
| 2024-01-04 | 50           | 500           |

### Additional Options:

- **`ROWS BETWEEN`** (optional): If you want to refine how the running total is calculated, you can use the `ROWS BETWEEN` clause within the `OVER()` clause to specify a sliding window of rows. For example, `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW` will sum all rows from the beginning up to the current row.

  ```sql
  SUM(sales_amount) OVER(
      ORDER BY date
      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS running_total
  ```

This clause allows you to control how many rows before or after the current row are included in the calculation. However, the default behavior without `ROWS` is already equivalent to this.

### Practical Usage:
- **Financial Reports**: Running totals are commonly used in financial reports to show cumulative revenue, expenses, or profits.
- **Inventory Management**: You might track cumulative stock levels over time.
- **Sales Analysis**: You can analyze cumulative sales over different periods.

By using window functions for running totals, you can create complex analytical queries that provide deeper insights into your data.

# Using a CASE statement inside the SUM() window function

## Calculate a running total of hot days (75F or higher)
![image](temp_table.png)

To complete the SQL query, you need to calculate a running total of "hot days" where the temperature (`temp_f`) is 75°F or higher. This can be done using a `CASE` statement inside the `SUM()` window function.

```sql
SELECT
    date,
    temp_f,
    COALESCE(SUM(CASE WHEN temp_f > 75 THEN 1 ELSE 0 END)
    OVER (ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW), 0)
    AS hot_days
FROM temp
LIMIT 5;
```

### Explanation:
- **`CASE WHEN temp_f > 75 THEN 1 ELSE 0 END`**: This `CASE` statement checks if the temperature is greater than 75°F. If true, it returns 1; otherwise, it returns 0. This helps in counting the hot days.
- **`SUM(...)`**: The `SUM` function calculates the running total of the values produced by the `CASE` statement.
- **`OVER (ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)`**: This clause defines the window over which the running total is calculated:
  - `ORDER BY date`: It orders the rows by the date.
  - `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`: This specifies that the sum should be calculated from the first row up to the current row.

This query will give you a cumulative count of hot days (days with temperatures above 75°F) as of each date, which matches the required output.

To ensure that the `hot_days` column displays `0` instead of a blank value, you can use the `COALESCE()` function. The `COALESCE()` function returns the first non-null expression among its arguments, which in this case will ensure that if the `SUM` function returns a `NULL`, it is replaced with `0`.

### Explanation:
- **`COALESCE(..., 0)`**: This will return the running total calculated by the `SUM()` function. If the `SUM()` returns `NULL` (which might happen in some databases if there are no qualifying rows for summation), it will instead return `0`.